In [1]:
%load_ext autoreload
%autoreload 2
import os; import sys; sys.path.insert(0,'../')
import pandas as pd
import tqdm
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import socceraction.atomic.vaep as vaep

In [2]:
## Configure file and folder names
datafolder = "../data"
spadl_h5 = os.path.join(datafolder,"spadl-statsbomb.h5")
predictions_h5 = os.path.join(datafolder,"predictions.h5")

In [3]:
games = pd.read_hdf(spadl_h5,"games")
games = games[games.competition_name == "FIFA World Cup"]
print("nb of games:", len(games))

nb of games: 64


In [4]:
players = pd.read_hdf(spadl_h5,"players")
teams = pd.read_hdf(spadl_h5,"teams")
actiontypes = pd.read_hdf(spadl_h5, "atomic_actiontypes")
bodyparts = pd.read_hdf(spadl_h5, "bodyparts")

A = []
for game in tqdm.tqdm(list(games.itertuples())):
    actions = pd.read_hdf(spadl_h5,f"atomic_actions/game_{game.game_id}")
    actions = (
        actions.merge(actiontypes,how="left")
        .merge(bodyparts,how="left")
        .merge(players,how="left")
        .merge(teams,how="left",)
        .sort_values(["game_id","period_id","action_id"])
        .reset_index(drop=True)
    )
    preds = pd.read_hdf(predictions_h5,f"game_{game.game_id}")
    values = vaep.value(actions,preds.scores,preds.concedes)
    A.append(pd.concat([actions,preds,values],axis=1))
A = pd.concat(A)#.sort_values(["game_id","period_id", "time_seconds", "timestamp"]).reset_index(drop=True)
A.columns

100%|██████████| 64/64 [00:05<00:00, 12.49it/s]


Index(['game_id', 'period_id', 'action_id', 'time_seconds', 'timestamp',
       'team_id', 'player_id', 'x', 'y', 'dx', 'dy', 'type_id', 'bodypart_id',
       'type_name', 'bodypart_name', 'player_name', 'player_nickname',
       'jersey_number', 'country_id', 'country_name', 'extra', 'team_name',
       'scores', 'concedes', 'offensive_value', 'defensive_value',
       'vaep_value'],
      dtype='object')

### Most valuable players in the world cup

In [30]:
A["player"] = A[["player_nickname","player_name"]].apply(lambda x: x[0] if x[0] else x[1],axis=1)

A["count"] = 1

A_ = A[~A.type_name.isin(["goal","out","penalty_shot","freekick_shot"])]#,"penalty_shot","freekick_shot","shot"])] # ignore penalties
players = (
    A_[["player_id","team_name","player","vaep_value","count"]]
    .groupby(["player_id","team_name","player"])
    .sum()
    .reset_index()
)
players = players.sort_values("vaep_value",ascending=False)
players[:20]

,player_id,team_name,player,vaep_value,count
122,4320.0,Brazil,Neymar,4.286853,1096
117,4269.0,Serbia,Aleksandar Mitrović,3.336891,273
6,3009.0,France,Kylian Mbappé,3.148116,713
92,3621.0,Belgium,Eden Hazard,2.921978,1016
44,3289.0,Belgium,Romelu Lukaku,2.917556,386
33,3233.0,England,Raheem Sterling,2.816944,560
89,3604.0,France,Olivier Giroud,2.760253,509
45,3293.0,England,Jesse Lingard,2.736933,778
209,5246.0,Uruguay,Luis Suárez,2.591835,505
599,20004.0,France,Paul Pogba,2.589124,1010


### Normalize for minutes played

In [32]:
pg = pd.read_hdf(spadl_h5,"player_games")
pg = pg[pg.game_id.isin(games.game_id)]
mp = pg[["player_id","minutes_played"]].groupby("player_id").sum().reset_index()
stats = players.merge(mp)
stats = stats[stats.minutes_played > 270]
stats["vaep_rating"] = stats.vaep_value * 90 / stats.minutes_played
stats.sort_values("vaep_rating",ascending=False)[:30]

,player_id,team_name,player,vaep_value,count,minutes_played,vaep_rating
1,4269.0,Serbia,Aleksandar Mitrović,3.336891,273,271,1.108192
0,4320.0,Brazil,Neymar,4.286853,1096,478,0.807148
10,5503.0,Argentina,Lionel Messi,2.471809,758,378,0.588526
5,3233.0,England,Raheem Sterling,2.816944,560,456,0.555976
30,3083.0,South Korea,Son Heung-Min,1.730315,334,288,0.540723
4,3289.0,Belgium,Romelu Lukaku,2.917556,386,488,0.538074
32,5619.0,South Korea,Cho Hyun-Woo,1.714494,145,288,0.535779
2,3009.0,France,Kylian Mbappé,3.148116,713,550,0.515146
28,5198.0,Spain,Diego Costa,1.791538,262,315,0.511868
42,3504.0,England,Ruben Loftus-Cheek,1.603062,462,285,0.506230
